# 통합함수만들기


In [11]:
def stockDataReader_fn(stock_name, start_date=None, end_date=None):
    import FinanceDataReader as fdr
    import datetime
    import os
    import pandas as pd

    # 1. 저장 파일 형식에 필요한 Data 불러오기
    today_year = datetime.datetime.today().year
    today_month = datetime.datetime.today().month
    today_day = datetime.datetime.today().day

    # 2. KRX 종목 DataFrame 존재 여부 확인
    csv_path = "krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)

    if os.path.isfile(csv_path):
        krx_df = pd.read_csv(csv_path)
    else:
        krx_df = fdr.StockListing("KRX")
        krx_df.to_csv(csv_path)

    # 3. 종목 코드 찾기
    if stock_name in krx_df["Name"].values:
        stock_symbol = krx_df[krx_df["Name"] == stock_name]["Code"].values[0]
    else:
        print("해당 종목이 없습니다.")
        raise "해당 종목이 없습니다."

    stock_df = fdr.DataReader(stock_symbol, start=start_date, end=end_date)
    return stock_df


def mdd_fn(df):
    df = df[["Close"]].copy()

    def return_fn(df):
        return df["Close"].pct_change().fillna(0)

    def cum_return_fn(df_return):
        return (1 + df_return).cumprod()

    df["Return"] = return_fn(df)
    df["CumReturn"] = cum_return_fn(df["Return"])
    df["MaxCumReturn"] = df["CumReturn"].cummax()
    df["DrawDown"] = (df["CumReturn"] / df["MaxCumReturn"]) - 1
    mdd = df["DrawDown"].min()
    list_info = []
    min_close_value = df["Close"].min()
    max_close_value = df["Close"].max()

    print(
        "최고: {}\n최저: {}\nMDD: {}".format(
            max_close_value, min_close_value, round(mdd * 100, 2)
        )
    )
    df_max_close = df[df["DrawDown"] == 0].copy()
    df_max_close.loc[df.index[len(df) - 1]] = 0
    period = df_max_close.index[1:] - df_max_close.index[:-1]
    mdd_days = period.days
    max_period = mdd_days.max()
    max_period_idx = mdd_days.argmax()
    print(
        "MAX Draw Down: {} ~ {}".format(
            df_max_close.index[:-1][max_period_idx].date(),
            df_max_close.index[1:][max_period_idx].date(),
        )
    )
    print("{} days".format(max_period))
    print("max period : {}".format(max_period))

    list_info.append(max_close_value)
    list_info.append(min_close_value)
    list_info.append(round(mdd * 100, 2))
    list_info.append(df_max_close.index[:-1][max_period_idx].date())
    list_info.append(df_max_close.index[1:][max_period_idx].date())
    list_info.append(max_period)

    return df, list_info


def macdOscillator_fn(df, short_N, long_N, signal_N):
    df = df[["Close"]].copy()

    df["Short"] = df["Close"].ewm(span=short_N, adjust=False).mean()
    df["Long"] = df["Close"].ewm(span=long_N, adjust=False).mean()
    df["MACD"] = df["Short"] - df["Long"]
    df["Signal"] = df["MACD"].ewm(span=signal_N, adjust=False).mean()
    df["MACD Oscillator"] = df["MACD"] - df["Signal"]

    return df[["MACD", "Signal", "MACD Oscillator"]]


def rsi_fn(df, rsi_period=14):
    import numpy as np

    df_close = df[["Close"]].copy()

    df_close["Change"] = df_close["Close"] - df_close["Close"].shift(1)
    df_close["Gain"] = np.where(df_close["Change"] >= 0, df_close["Change"], 0)
    df_close["Loss"] = np.where(df_close["Change"] < 0, df_close["Change"] * -1, 0)
    df_close["avgGain"] = df_close["Gain"].rolling(window=rsi_period).mean()
    df_close["avgLoss"] = df_close["Loss"].rolling(window=rsi_period).mean()
    df_close["RS"] = df_close["avgGain"] / df_close["avgLoss"]
    df_close["RSI"] = 100 - (100 / (1 + df_close["RS"]))

    return df_close


def traningRoom1_fn(
    stock_name,
    start_date=None,
    end_date=None,
    short_N=9,
    long_N=26,
    signal_N=13,
    rsi_period=14,
):
    df = stockDataReader_fn(stock_name, start_date, end_date)
    df_dd, stock_info = mdd_fn(df)
    df_macd = macdOscillator_fn(df, short_N, long_N, signal_N)
    df_rsi = rsi_fn(df, rsi_period)

    import plotly.graph_objects as go

    from plotly import subplots

    candle = go.Candlestick(
        x=df.index,
        open=df["Open"],
        high=df["High"],
        low=df["Low"],
        close=df["Close"],
        name="Candle Chart",
    )
    dd = go.Scatter(
        x=df.index, y=df_dd["DrawDown"] * 100, name="DrawDown", fill="tozeroy"
    )
    macd = go.Scatter(x=df.index, y=df_macd["MACD"], name="MACD")
    signal = go.Scatter(x=df.index, y=df_macd["Signal"], name="Signal")
    oscillator = go.Scatter(
        x=df.index, y=df_macd["MACD Oscillator"], name="MACD Oscillator"
    )
    rsi = go.Scatter(x=df.index, y=df_rsi["RSI"], name="RSI")
    volume = go.Bar(x=df.index, y=df["Volume"], name="Volume")

    fig = subplots.make_subplots(
        rows=5,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.01,
        subplot_titles=(
            "<b>{}</b>".format(stock_name),
            "<b>DrawDown</b>",
            "<b>MACD</b>",
            "<b>RSI</b>",
            "<b>Volume</b>",
        ),
    )

    fig.append_trace(candle, row=1, col=1)
    fig.append_trace(dd, row=2, col=1)
    fig.append_trace(macd, row=3, col=1)
    fig.append_trace(signal, row=3, col=1)
    fig.append_trace(oscillator, row=3, col=1)
    fig.append_trace(rsi, row=4, col=1)

    fig.add_trace(
        go.Scatter(
            x=df_rsi.index,
            y=(df_rsi["RSI"] / df_rsi["RSI"]).fillna(1) * 30,
            line=dict(color="red", dash="dash"),
            name="Low bound",
        ),
        row=4,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=df_rsi.index,
            y=(df_rsi["RSI"] / df_rsi["RSI"]).fillna(1) * 70,
            line=dict(color="red", dash="dash"),
            name="High bound",
        ),
        row=4,
        col=1,
    )

    fig.append_trace(volume, row=5, col=1)

    fig.update_xaxes(rangeslider_visible=False)
    fig.update_layout(width=1000, height=1300)
    fig.update_layout(
        legend_title_text="Content",
        title="{}, Max: {}, Min: {}, MDD: {}%, Period: {} ~ {}".format(
            stock_name,
            stock_info[0],
            stock_info[1],
            stock_info[2],
            stock_info[3],
            stock_info[4],
        ),
    )
    fig.update_yaxes(tickformat=",")
    fig.show()


In [12]:
stocking_name = "삼성전자"
start_date = "2019"
end_date = "2021"
short_N = 9
long_N = 26
signal_N = 13
rsi_period = 14

traningRoom1_fn(
    stocking_name,
    start_date,
    end_date,
    short_N,
    long_N,
    signal_N,
    rsi_period,
)


최고: 81000
최저: 37450
MDD: -31.89
MAX Draw Down: 2020-01-20 ~ 2020-11-13
298 days
max period : 298


In [13]:
stocking_name = "LG전자"

traningRoom1_fn(
    stocking_name,
    start_date,
    end_date,
    short_N,
    long_N,
    signal_N,
    rsi_period,
)

최고: 135000
최저: 41850
MDD: -49.33
MAX Draw Down: 2019-06-11 ~ 2020-08-10
426 days
max period : 426


In [14]:
stocking_name = "SK하이닉스"

traningRoom1_fn(
    stocking_name,
    start_date,
    end_date,
    short_N,
    long_N,
    signal_N,
    rsi_period,
)

최고: 120500
최저: 57700
MDD: -34.29
MAX Draw Down: 2020-02-17 ~ 2020-12-02
289 days
max period : 289
